# Разметка на основе правил

In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')

In [ ]:
# Определение набора положительных и отрицательных слов
positive_words = ['happy', 'excited', 'awesome']
negative_words = ['disappointed', 'frustrated', 'terrible']

In [ ]:
# Функция для присвоения метки настроения на основе наличия положительных или отрицательных слов
def get_sentiment(text):
    # Токенизация текста на отдельные слова
    words = nltk.word_tokenize(text.lower())
    # Подсчет количества положительных и отрицательных слов
    num_positive = sum([1 for word in words if word in positive_words])
    num_negative = sum([1 for word in words if word in negative_words])
    # Присвоение метки на основе чистой оценки настроения
    if num_positive > num_negative:
        return 'positive'
    elif num_negative > num_positive:
        return 'negative'
    else:
        return 'neutral'

In [ ]:
# Тестирование функции
text1 = "I'm so happy today!"
text2 = "I'm so frustrated with this service."
text3 = "This movie was okay, I guess."
print(get_sentiment(text1))
print(get_sentiment(text2))
print(get_sentiment(text3))

In [ ]:
# Загрузка датасета твитов
tweets = pd.read_csv('tweets.csv')

In [ ]:
tweets.head()

In [ ]:
from textblob import TextBlob

In [ ]:
# Определение функции для получения полярности настроения твита
def get_sentiment(tweet):
    blob = TextBlob(tweet)
    return blob.sentiment.polarity

In [ ]:
# Применение функции к столбцу 'text' датасета для получения полярности настроения каждого твита
tweets['sentiment'] = tweets['text'].apply(get_sentiment)

In [ ]:
tweets.head()

In [ ]:
# Определение функции для присвоения метки настроения на основе оценки полярности настроения
def get_sentiment_label(score):
    if score > 0:
        return 'positive'
    elif score < 0:
        return 'negative'
    else:
        return 'neutral'

In [ ]:
tweets.head()

In [ ]:
# Применение функции к столбцу 'sentiment' датасета для присвоения меток настроения
tweets['sentiment_label'] = tweets['sentiment'].apply(get_sentiment_label)

In [ ]:
tweets.head()

In [ ]:
# Определение функции для присвоения метки на основе числового порога
def get_category(value):
    if value > 100000:
        return 'high'
    elif value < 5000:
        return 'low'
    else:
        return 'medium'

In [ ]:
# Применение функции к столбцу данных
tweets['follow_category'] = tweets['user_followers'].apply(get_category)

In [ ]:
tweets.head()

In [ ]:
tweets['follow_category'].hist()

# Разметка данных с использованием активного обучения

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

In [ ]:
# Загрузка датасета рецензий на фильмы
reviews = pd.read_csv('movie.csv', engine='python', error_bad_lines=False)

In [ ]:
reviews.head()

In [ ]:
# Замена 'neg' на 0 и 'pos' на 1
reviews['label'] = reviews['label'].replace({'neg': 0, 'pos': 1})

In [ ]:
reviews.head()

In [ ]:
reviews['label'].hist()

In [ ]:
# Перемешивание данных
reviews = shuffle(reviews)

In [ ]:
# Разделение датасета на размеченную и неразмеченную выборки
labeled_reviews, unlabeled_reviews = train_test_split(reviews, train_size=0.2, random_state=42)

In [ ]:
labeled_reviews.head()

In [ ]:
unlabeled_reviews.head()

In [ ]:
# Определение функции для обучения модели логистической регрессии на размеченных данных
def train_model(labeled_reviews):
    # Векторизация текстовых данных с помощью TF-IDF
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(labeled_reviews['text'])
    y = labeled_reviews['label']

    # Обучение модели логистической регрессии на размеченных данных
    model = LogisticRegression()
    model.fit(X, y)

    return model, vectorizer

In [ ]:
# Обучение начальной модели на небольшом наборе 
model, vectorizer = train_model(labeled_reviews)

In [ ]:
# Использование исходной модели для прогнозирования настроения неразмеченных данных
X_unlabeled = vectorizer.transform(unlabeled_reviews['text'])
y_unlabeled_predicted = model.predict(X_unlabeled)

In [ ]:
# Вычислить неопределенности или энтропии предсказаний
y_unlabeled_proba = model.predict_proba(X_unlabeled)
uncertainty = -(y_unlabeled_proba * np.log2(y_unlabeled_proba)).sum(axis=1)

In [ ]:
# Выбор 100 наиболее неопределенных точек данных для маркировки человеком
labeled_reviews_new = unlabeled_reviews.iloc[uncertainty.argsort()[:100]]
unlabeled_reviews_new = unlabeled_reviews.iloc[uncertainty.argsort()[100:]]

In [ ]:
# Разметка новых точек данных и добавление их к размеченному множеству
labeled_reviews = pd.concat([labeled_reviews, labeled_reviews_new])

In [ ]:
# Переобучение модели на расширенном маркированном множестве
model, vectorizer = train_model(labeled_reviews)

In [ ]:
reviews_test = pd.read_csv('movie.csv', engine='python', error_bad_lines=False)

In [ ]:
# Замена 'neg' на 0 и 'pos' на 1 в тестовом датасете
reviews_test['label'] = reviews_test['label'].replace({'neg': 0, 'pos': 1})

In [ ]:
# Оценка модели на тестовом датасете
X_test = vectorizer.transform(reviews_test['text'])
y_test_predicted = model.predict(X_test)
f1 = f1_score(reviews_test['label'], y_test_predicted)

print(f1)